# System recommendation 

In [1]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_percentage_error
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
#import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
# Import Halving Grid Search
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
import xgboost as xgb
from scipy.stats import kurtosis,skew
from numpy import mean,sqrt,square
from scipy.fftpack import fft, fftfreq
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import PredefinedSplit
import pickle
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
tag_select=['min_water_3',# 1
 'min_solid percentage_10',# 2
 'HH TPH',# 3
 'max_delta LL charge cell_10',# 4
 'power',# 5
 'rms_delta LL charge cell_3',# 6
 'var_delta HH charge cell_10',# 7
 'max_covelin law_10',# 8
 'LL charge cell_(t-2)',# 9
 'min_granulometry_5',# 10
 'max_bornite law_10', # 11
 'min_charge cell_5',# 12
 'chalcocite law_(t-2)',# 13
 'max_sag power index_5',# 14
 'min_speed_3',# 15
 'var_bornite law_3',# 16
 'var_speed_3',# 17
 'min_pyrite law_10', # 18
 'crusher index_(t-5)', # 19
 'var_power_3',# 20
 'var_chalcocite law_3',# 21
 'var_bornite law_5',# 22
 'var_solid percentage_3',# 23
 'var_speed_10',# 24
 'ball work index_(t-1)',# 25
 'var_chalcocite law_5', # 26
 'var_water_3',# 27
 'chalcopyrite law_(t-5)',# 28
 'var_crusher index_10',# 29
 'var_chalcopyrite law_3',# 30
 'var_granulometry_3',# 31
 'var_delta HH charge cell_3'# 32
 ]

In [3]:
def pipeline_feature_engineering(df):
      
    df_vars = df.copy()
    
    pipe_df_features=pd.DataFrame()

    pipe_df_features['HH charge cell']=df_vars["HH charge cell"] ##
    pipe_df_features['LL charge cell']=df_vars["LL charge cell"] ##
    pipe_df_features['charge cell']=df_vars["charge cell"] ##

    pipe_df_features["delta HH charge cell"]=pipe_df_features["HH charge cell"]-pipe_df_features["charge cell"] ##
    pipe_df_features["delta LL charge cell"]=pipe_df_features["charge cell"]-pipe_df_features["LL charge cell"] ##

    pipe_df_features["min_water_3"]=df_vars["water"].rolling(window=3).apply(lambda x:np.nanmin(list(x))) # 1
    pipe_df_features["min_solid percentage_10"]=df_vars["solid percentage"].rolling(window=10).apply(lambda x:np.nanmin(list(x))) # 2
    pipe_df_features['HH TPH']=df_vars['HH TPH'] # 3
    pipe_df_features["max_delta LL charge cell_10"]=pipe_df_features["delta LL charge cell"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 4
    pipe_df_features['power']=df_vars['power'] # 5
    pipe_df_features["rms_delta LL charge cell_3"]=pipe_df_features["delta LL charge cell"].rolling(window=3).apply(lambda x: sqrt(mean(square(list(x))))) # 6
    pipe_df_features["var_delta HH charge cell_10"]=pipe_df_features["delta HH charge cell"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 7
    pipe_df_features[f"max_covelin law_10"]=df_vars["covelin law"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 8
    pipe_df_features[f'LL charge cell_(t-2)'] = pipe_df_features["LL charge cell"].shift(2) # 9
    pipe_df_features[f"min_granulometry_5"]=df_vars["granulometry"].rolling(window=5).apply(lambda x:np.nanmin(list(x))) # 10
    pipe_df_features[f"max_bornite law_10"]=df_vars["bornite law"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) # 11
    pipe_df_features[f"min_charge cell_5"]=pipe_df_features["charge cell"].rolling(window=5).apply(lambda x:np.nanmin(list(x))) # 12
    pipe_df_features[f'chalcocite law_(t-2)'] = df_vars["chalcocite law"].shift(2) # 13
    pipe_df_features[f"max_sag power index_5"]=df_vars["sag power index"].rolling(window=5).apply(lambda x:np.nanmax(list(x))) # 14
    pipe_df_features[f"min_speed_3"]=df_vars["speed"].rolling(window=3).apply(lambda x:np.nanmin(list(x))) # 15
    pipe_df_features[f"var_bornite law_3"]=df_vars["bornite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 16
    pipe_df_features[f"var_speed_3"]=df_vars["speed"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 17
    pipe_df_features[f"min_pyrite law_10"]=df_vars["pyrite law"].rolling(window=10).apply(lambda x:np.nanmin(list(x))) # 18
    pipe_df_features[f'crusher index_(t-5)'] = df_vars["crusher index"].shift(5) # 19
    pipe_df_features[f"var_power_3"]=df_vars["power"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 20
    pipe_df_features[f"var_chalcocite law_3"]=df_vars["chalcocite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 21
    pipe_df_features[f"var_bornite law_5"]=df_vars["bornite law"].rolling(window=5).apply(lambda x: np.nanvar(x)) # 22
    pipe_df_features[f"var_solid percentage_3"]=df_vars["solid percentage"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 23
    pipe_df_features[f"var_speed_10"]=df_vars["speed"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 24
    pipe_df_features[f"ball work index_(t-1)"] = df_vars["ball work index"].shift(1) # 25
    pipe_df_features[f"var_chalcocite law_5"]=df_vars["chalcocite law"].rolling(window=5).apply(lambda x: np.nanvar(x)) # 26
    pipe_df_features[f"var_water_3"]=df_vars["water"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 27
    pipe_df_features[f'chalcopyrite law_(t-5)'] = df_vars["chalcopyrite law"].shift(5) # 28
    pipe_df_features[f"var_crusher index_10"]=df_vars["crusher index"].rolling(window=10).apply(lambda x: np.nanvar(x)) # 29
    pipe_df_features[f"var_chalcopyrite law_3"]=df_vars["chalcopyrite law"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 30
    pipe_df_features[f"var_granulometry_3"]=df_vars["granulometry"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 31
    pipe_df_features[f"var_delta HH charge cell_3"]=pipe_df_features["delta HH charge cell"].rolling(window=3).apply(lambda x: np.nanvar(x)) # 32
    
    
    return pipe_df_features

In [4]:
def feature_engineering_recommendation(df):
      
    pipe_df_features=df.copy()
    
    #pipe_df_features['HH charge cell']=df_vars["HH charge cell"] ##
    #pipe_df_features['LL charge cell']=df_vars["LL charge cell"] ##
    #pipe_df_features['charge cell']=df_vars["charge cell"] ##

    pipe_df_features["delta HH charge cell"]=pipe_df_features["HH charge cell"]-pipe_df_features["charge cell"] ##
    pipe_df_features["delta LL charge cell"]=pipe_df_features["charge cell"]-pipe_df_features["LL charge cell"] ##
    

    pipe_df_features["max_delta LL charge cell_10"]=pipe_df_features["delta LL charge cell"].rolling(window=10).apply(lambda x:np.nanmax(list(x))) ##
    pipe_df_features["rms_delta LL charge cell_3"]=pipe_df_features["delta LL charge cell"].rolling(window=3).apply(lambda x: sqrt(mean(square(list(x))))) ##
    pipe_df_features["var_delta HH charge cell_10"]=pipe_df_features["delta HH charge cell"].rolling(window=10).apply(lambda x: np.nanvar(x)) ##
    pipe_df_features[f'LL charge cell_(t-2)'] = pipe_df_features["LL charge cell"].shift(2) ##
    pipe_df_features[f"min_charge cell_5"]=pipe_df_features["charge cell"].rolling(window=5).apply(lambda x:np.nanmin(list(x))) ##
    pipe_df_features[f"var_delta HH charge cell_3"]=pipe_df_features["delta HH charge cell"].rolling(window=3).apply(lambda x: np.nanvar(x)) ##
    
    return pipe_df_features

In [5]:
variables_interes=['pyrite law', 'chalcopyrite law',
       'chalcocite law', 'covelin law', 'crusher index', 'sag power index',
       'ball work index', 'bornite law','charge cell', 'speed','HH TPH', 'granulometry','Edad','loss of TPH',"solid percentage","HH charge cell","LL charge cell","TPH",
       #"recommendation base model carl","recommendation base model acn",
       "water","power"]

In [6]:
# read df full csv
df_features=pd.read_csv('../../../data/processed data/df_feature_engineering_18December2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')#[variables_interes]#["2020":"2021"]
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

(859335, 403)

# Load pipeline and XGBoost

In [7]:
# Load
model= xgb.XGBRegressor()
model.load_model("../../../models/model_xgb.bin")

In [8]:
# Leer pipeline:
pipe= pickle.load(open('../../../models/pipe_xgb.pkl', 'rb'))

# Recommendation base model ACN

In [9]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//..//models//pickles//Modelo update Accenture//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//..//models//pickles//Modelo update Accenture//cc_recommendations_prod.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK['s'+subcontexto].loc[granulometria]["cc"]
  
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [10]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [11]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model acn"]=Rec_HH_CC

# Recommendation base model Carl

In [12]:
# Se carga clasificador de mineralogia
kmeans = pickle.load(open('..//..//..//models//pickles//Clasificador_dispatch_n_4.pkl', 'rb'))
#kmeans.feature_names_in_
# Se carga la curva de celda de carga v/s granulometria (dado un subcontexto)
dicK = pickle.load(open('..//..//..//models//pickles//Curvas de celda de carga vs granulometria productivo actualizado.pkl', 'rb'))

# Recomendación 
def recommendationcc(granulometria,edad_sag,cluster):
 
    # Generación del subcontexto
    subcontexto = str(int(edad_sag)) + str(int(cluster))
   
    # Obtención de la recomendación
    consejo=dicK[subcontexto].loc[granulometria]["cc"]
    
    return consejo #pd.DataFrame({"recommendation":list(consejo)})

In [13]:
# Predict cluster
df_features["cluster"]=kmeans.predict(df_features[['crusher index',
                                              'sag power index',
                                               'ball work index',
                                              'chalcopyrite law',
                                              'pyrite law',
                                              'chalcocite law',
                                              'bornite law',
                                              'covelin law']])

In [14]:
# Recomendación 
Rec_HH_CC=df_features.apply(lambda row: recommendationcc(
                                              row["granulometry"],
                                            row["Edad"],
                                            row["cluster"]
                                                    ),axis=1)

# Columna de recomendación
df_features["recommendation base model carl"]=Rec_HH_CC

# Data test para recomendación HH CC

In [17]:
# Test
test=pd.concat([df_features["2020-04"],df_features["2021-09"],df_features["2022"]])
test.head(1)

,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,TPH,water,speed,HH TPH,granulometry,Edad,loss of TPH,power,solid percentage,HH charge cell,LL charge cell,delta HH TPH,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_water_10,max_water_10,min_water_10,rms_water_10,var_water_10,mean_water_5,max_water_5,min_water_5,rms_water_5,var_water_5,mean_water_3,max_water_3,min_water_3,rms_water_3,var_water_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_power_10,max_power_10,min_power_10,rms_power_10,var_power_10,mean_power_5,max_power_5,min_power_5,rms_power_5,var_power_5,mean_power_3,max_power_3,min_power_3,rms_power_3,var_power_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,va

In [18]:
# Ytest reg
Ytest=test["TPH"]
Xtest=test.drop(columns=["TPH"])

# Simulation time series predict

# 2020

In [ ]:
# test 2020
test_time_serie=Xtest["2020"]
test_time_serie["TPH"]=Ytest["2020"]

features=pipeline_feature_engineering(test_time_serie)

Xtest1=pipe.transform(features)
Ypred= model.predict(Xtest1)

test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))

test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# 2021

In [ ]:
# test 2021
test_time_serie=Xtest["2021"]
test_time_serie["TPH"]=Ytest["2021"]

features=pipeline_feature_engineering(test_time_serie)

Xtest1=pipe.transform(features)
Ypred= model.predict(Xtest1)

test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))

test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=7, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['water'], ##FF6511
                    mode='lines',
                    name='water',line=dict(width=3, ),legendgroup = '1'),row=7, col=1)  
#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1000, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# 2022

In [ ]:
# test 2022
test_time_serie=Xtest["2022"]
test_time_serie["TPH"]=Ytest["2022"]

features=pipeline_feature_engineering(test_time_serie)

Xtest1=pipe.transform(features)
Ypred= model.predict(Xtest1)

test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))

test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

In [ ]:
titulos=tuple(["TPH & HH TPH", "charge cell","granulometry","SPI","speed","solid percentage"])
fig = make_subplots(
    rows=6, cols=1,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['TPH predict'], ##FF6511
                    mode='lines',
                    name="TPH predict",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

#fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
#                    mode='lines',
#                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1) 

fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)   

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=6, col=1)  


#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# recommendation function

In [19]:
range_hh_cc=np.arange(660,1050,25)
range_hh_cc

array([ 660,  685,  710,  735,  760,  785,  810,  835,  860,  885,  910,
        935,  960,  985, 1010, 1035])

In [27]:
def tph_function(df,HH_CC):

    df_rec=df.copy()
    df_rec["HH charge cell"]=HH_CC
    df_rec["LL charge cell"]=HH_CC-80
    df_rec["charge cell"]=(df_rec["HH charge cell"]+df_rec["LL charge cell"])/2
    df_rec_features=feature_engineering_recommendation(df_rec).iloc[[-1]]
    #display(df_rec_features)
    X=pipe.transform(df_rec_features)
    Ypred= model.predict(X)[0]

    return Ypred


In [28]:
def optimum_recommendation(df,range_hh_cc):
    list_TPH=np.array([tph_function(df,i) for i in range_hh_cc])
    index_max=np.argmax(list_TPH)
    rec_hh_cc=range_hh_cc[index_max]
    tph_opt=list_TPH[index_max]
    
    df_tph=pd.DataFrame({"TPH":list_TPH,"HH CC":range_hh_cc})
    

    # Create traces
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=df_tph["HH CC"], y=df_tph["TPH"],
                        mode='lines+markers',
                        name='TPH'))

    fig.update_layout(height=500, width=1200, title_text="TPH vs HH charge cell" ,xaxis_title="HH charge cell",
        yaxis_title="TPH")   

    fig.update_layout(hovermode="x unified")                

    fig.show()

    return rec_hh_cc,tph_opt

In [29]:
# test 2021
test_time_serie=Xtest["2021"]
test_time_serie["TPH"]=Ytest["2021"]

#features=pipeline_feature_engineering(test_time_serie)

features=test_time_serie
Xtest1=pipe.transform(features)
Ypred= model.predict(Xtest1)

test_time_serie["TPH predict"]=Ypred


# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))

test_time_serie.dropna(inplace=True)
data=test_time_serie
data.head(1)

R2:  0.942024691353748 
 RMSE:  106.27146668896225


,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,water,speed,HH TPH,granulometry,Edad,loss of TPH,power,solid percentage,HH charge cell,LL charge cell,delta HH TPH,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_water_10,max_water_10,min_water_10,rms_water_10,var_water_10,mean_water_5,max_water_5,min_water_5,rms_water_5,var_water_5,mean_water_3,max_water_3,min_water_3,rms_water_3,var_water_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_power_10,max_power_10,min_power_10,rms_power_10,var_power_10,mean_power_5,max_power_5,min_power_5,rms_power_5,var_power_5,mean_power_3,max_power_3,min_power_3,rms_power_3,var_power_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_so

In [30]:
data1=data.loc['2021-09-15 03:10:00':'2021-09-15 03:59:00']
data1.head()

,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,water,speed,HH TPH,granulometry,Edad,loss of TPH,power,solid percentage,HH charge cell,LL charge cell,delta HH TPH,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_water_10,max_water_10,min_water_10,rms_water_10,var_water_10,mean_water_5,max_water_5,min_water_5,rms_water_5,var_water_5,mean_water_3,max_water_3,min_water_3,rms_water_3,var_water_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_power_10,max_power_10,min_power_10,rms_power_10,var_power_10,mean_power_5,max_power_5,min_power_5,rms_power_5,var_power_5,mean_power_3,max_power_3,min_power_3,rms_power_3,var_power_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_so

In [31]:
list_df_recommendation=[]
list_df_tph=[]
list_index=[]
for i in data1.index:
    try:
        inicio=i
        print(inicio)
        fin=inicio+timedelta(minutes=10)
        
        df=data1.loc[inicio:fin]
        #display(df)
        rec,tph=optimum_recommendation(df,range_hh_cc)

        list_df_recommendation.append(rec)
        list_df_tph.append(tph)
        list_index.append(fin)
        
    except Exception as e:
        print(e)
        #print("Error :",i)
        #display(df)

2021-09-15 03:10:00


2021-09-15 03:11:00


2021-09-15 03:12:00


2021-09-15 03:13:00


2021-09-15 03:14:00


...

2021-09-15 03:55:00


2021-09-15 03:56:00


2021-09-15 03:57:00


2021-09-15 03:58:00


2021-09-15 03:59:00


In [32]:
df_rec=pd.DataFrame({"Timestamp":list_index,"recommended HH charge cell":list_df_recommendation,"TPH optimum":list_df_tph}).set_index("Timestamp")
min_rec=df_rec["recommended HH charge cell"].quantile(0.25)
max_rec=df_rec["recommended HH charge cell"].quantile(0.75)
df_rec["soft recommended HH charge cell"]=df_rec.apply(lambda x: x["recommended HH charge cell"] if (x["recommended HH charge cell"]>=min_rec) and (x["recommended HH charge cell"]<=max_rec) else np.nan,axis=1).interpolate()
df_rec["soft recommended HH charge cell"]=df_rec["soft recommended HH charge cell"].rolling(window=10,min_periods=1).apply(np.nanmean)
df_rec

,recommended HH charge cell,TPH optimum,soft recommended HH charge cell
Timestamp,,,
2021-09-15 03:20:00,660,4488.147461,660.000000
2021-09-15 03:21:00,660,4486.420898,660.000000
2021-09-15 03:22:00,660,4474.187988,660.000000
2021-09-15 03:23:00,660,4442.299316,660.000000
2021-09-15 03:24:00,660,4427.002441,660.000000
2021-09-15 03:25:00,660,4475.858887,660.000000
2021-09-15 03:26:00,935,4347.796875,699.285714
2021-09-15 03:27:00,935,4428.176270,728.750000
2021-09-15 03:28:00,910,4514.989746,748.888889


In [ ]:
df_rec.to_csv("recommendation ML model.csv")

In [33]:
df_visualization=data1.merge(df_rec, left_index=True, right_index=True,how='left',suffixes=('_x', '_y'))
#df_visualization["recommended HH charge cell"]=df_visualization["recommended HH charge cell"].rolling(window=10,min_periods=1).apply(np.nanmean)
df_visualization.head()

,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,charge cell,water,speed,HH TPH,granulometry,Edad,loss of TPH,power,solid percentage,HH charge cell,LL charge cell,delta HH TPH,delta HH charge cell,delta LL charge cell,mean_pyrite law_10,max_pyrite law_10,min_pyrite law_10,rms_pyrite law_10,var_pyrite law_10,mean_pyrite law_5,max_pyrite law_5,min_pyrite law_5,rms_pyrite law_5,var_pyrite law_5,mean_pyrite law_3,max_pyrite law_3,min_pyrite law_3,rms_pyrite law_3,var_pyrite law_3,mean_chalcopyrite law_10,max_chalcopyrite law_10,min_chalcopyrite law_10,rms_chalcopyrite law_10,var_chalcopyrite law_10,mean_chalcopyrite law_5,max_chalcopyrite law_5,min_chalcopyrite law_5,rms_chalcopyrite law_5,var_chalcopyrite law_5,mean_chalcopyrite law_3,max_chalcopyrite law_3,min_chalcopyrite law_3,rms_chalcopyrite law_3,var_chalcopyrite law_3,mean_chalcocite law_10,max_chalcocite law_10,min_chalcocite law_10,rms_chalcocite law_10,var_chalcocite law_10,mean_chalcocite law_5,max_chalcocite law_5,min_chalcocite law_5,rms_chalcocite law_5,var_chalcocite law_5,mean_chalcocite law_3,max_chalcocite law_3,min_chalcocite law_3,rms_chalcocite law_3,var_chalcocite law_3,mean_covelin law_10,max_covelin law_10,min_covelin law_10,rms_covelin law_10,var_covelin law_10,mean_covelin law_5,max_covelin law_5,min_covelin law_5,rms_covelin law_5,var_covelin law_5,mean_covelin law_3,max_covelin law_3,min_covelin law_3,rms_covelin law_3,var_covelin law_3,mean_crusher index_10,max_crusher index_10,min_crusher index_10,rms_crusher index_10,var_crusher index_10,mean_crusher index_5,max_crusher index_5,min_crusher index_5,rms_crusher index_5,var_crusher index_5,mean_crusher index_3,max_crusher index_3,min_crusher index_3,rms_crusher index_3,var_crusher index_3,mean_sag power index_10,max_sag power index_10,min_sag power index_10,rms_sag power index_10,var_sag power index_10,mean_sag power index_5,max_sag power index_5,min_sag power index_5,rms_sag power index_5,var_sag power index_5,mean_sag power index_3,max_sag power index_3,min_sag power index_3,rms_sag power index_3,var_sag power index_3,mean_ball work index_10,max_ball work index_10,min_ball work index_10,rms_ball work index_10,var_ball work index_10,mean_ball work index_5,max_ball work index_5,min_ball work index_5,rms_ball work index_5,var_ball work index_5,mean_ball work index_3,max_ball work index_3,min_ball work index_3,rms_ball work index_3,var_ball work index_3,mean_bornite law_10,max_bornite law_10,min_bornite law_10,rms_bornite law_10,var_bornite law_10,mean_bornite law_5,max_bornite law_5,min_bornite law_5,rms_bornite law_5,var_bornite law_5,mean_bornite law_3,max_bornite law_3,min_bornite law_3,rms_bornite law_3,var_bornite law_3,mean_charge cell_10,max_charge cell_10,min_charge cell_10,rms_charge cell_10,var_charge cell_10,mean_charge cell_5,max_charge cell_5,min_charge cell_5,rms_charge cell_5,var_charge cell_5,mean_charge cell_3,max_charge cell_3,min_charge cell_3,rms_charge cell_3,var_charge cell_3,mean_water_10,max_water_10,min_water_10,rms_water_10,var_water_10,mean_water_5,max_water_5,min_water_5,rms_water_5,var_water_5,mean_water_3,max_water_3,min_water_3,rms_water_3,var_water_3,mean_speed_10,max_speed_10,min_speed_10,rms_speed_10,var_speed_10,mean_speed_5,max_speed_5,min_speed_5,rms_speed_5,var_speed_5,mean_speed_3,max_speed_3,min_speed_3,rms_speed_3,var_speed_3,mean_granulometry_10,max_granulometry_10,min_granulometry_10,rms_granulometry_10,var_granulometry_10,mean_granulometry_5,max_granulometry_5,min_granulometry_5,rms_granulometry_5,var_granulometry_5,mean_granulometry_3,max_granulometry_3,min_granulometry_3,rms_granulometry_3,var_granulometry_3,mean_power_10,max_power_10,min_power_10,rms_power_10,var_power_10,mean_power_5,max_power_5,min_power_5,rms_power_5,var_power_5,mean_power_3,max_power_3,min_power_3,rms_power_3,var_power_3,mean_solid percentage_10,max_solid percentage_10,min_solid percentage_10,rms_solid percentage_10,var_so

In [34]:
titulos=tuple(["TPH & HH TPH","speed", "charge cell","solid percentage","granulometry & SPI","water"])
fig = make_subplots(
    rows=3, cols=2,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True
    )


fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["TPH optimum"],
                    mode='lines',
                    name='TPH predict opt'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["TPH predict"],
                    mode='lines',
                    name='TPH predict'),row=1, col=1)
fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["HH TPH"],
                    mode='lines',
                    name='HH TPH'),row=1, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["TPH"],
                    mode='lines',
                    name='TPH'),row=1, col=1)

##
fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["HH charge cell"],
                    mode='lines',
                    name='HH charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['recommendation base model acn'],
                    mode='lines',
                    name='recommendation base model acn'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['recommendation base model carl'],
                    mode='lines',
                    name='recommendation base model carl'),row=2, col=1)


fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["soft recommended HH charge cell"],
                    mode='lines',
                    name='recommended HH charge cell ML model'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["soft recommended HH charge cell"]-80,
                    mode='lines',
                    name='recommended LL charge cell ML model'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["LL charge cell"],
                    mode='lines',
                    name='LL charge cell'),row=2, col=1)

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization["charge cell"],
                    mode='lines',
                    name='charge cell'),row=2, col=1)   

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=3, col=1)  


fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  



fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  

fig.add_trace(go.Scatter(x=df_visualization.index, y=df_visualization['water'], ##FF6511
                    mode='lines',
                    name='water',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  
#for i in range(1,6):
#   for j in range(1,4):
#        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.update_layout(height=1000, width=1700, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()